<a href="https://colab.research.google.com/github/FarhanDhanani/joker-clef-22-FAST-MT/blob/main/JOKER_CLEF_TASK_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center> This notebook is prepared to submit the solutions to the JOKER CLEF conference from the FAST-MT Team.</center></h1>

<center><img src="https://upload.wikimedia.org/wikipedia/en/e/e4/National_University_of_Computer_and_Emerging_Sciences_logo.png"/></center>

A simple set of booleans that represents the notebook is running on google colab or locally. And the model are already downloaded and loaded either on G-Drive or locally or it needs to be downloaded from the checkpoint repository.

In [1]:
'''
## A set of simple booleeans:
##  - (run_on_colab) = True : if notebook is running on colab
##  - (run_on_colab) = False : if notebook is running locally
'''
run_on_colab = True

'''
##  - (download_helsinki_NLP_for_translation_generation) = True : if Helsinki_nlp Model needs to get download from given checkpoint
##  - (download_helsinki_NLP_for_translation_generation) = False : if Helsinki_nlp Model is already downloaded and its path is step
'''
download_helsinki_NLP_for_translation_generation = True

'''
##  - (download_google_T5_base_for_translation_generation) = True : if Google_T5_base Model needs to get download from given checkpoint
##  - (download_google_T5_base_for_translation_generation) = False : if Google_T5_base Model is already downloaded and its path is step
'''
download_google_T5_base_for_translation_generation = True

'''
##  - (download_google_T5_small_for_translation_generation) = True : if Google_T5_small Model needs to get download from given checkpoint
##  - (download_google_T5_small_for_translation_generation) = False : if Google_T5_small Model is already downloaded and its path is step
'''
download_google_T5_small_for_translation_generation = True


'''
##  - (download_google_T5_large_for_translation_generation) = True : if Google_T5_large Model needs to get download from given checkpoint
##  - (download_google_T5_large_for_translation_generation) = False : if Google_T5_large Model is already downloaded and its path is step
'''
download_google_T5_large_for_translation_generation = True


Installing required dependencies and setuping base paths

In [2]:
if run_on_colab:
    from google.colab import drive
    from google.colab import files
    base_path = '/content/drive'
    drive.mount(base_path)
    base_path = base_path + '/My Drive/'

    ## UPLOADING REQUIREMENTS.TXT
    uploaded = files.upload()
    for fn in uploaded.keys():
      print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

    ## INSTALLING ALL DEPENDENCIES
    !pip3 install -r requirements_task3.txt

else:
    base_path = '/Users/fdhanani/Desktop/JOKER CLEF/'
    !pip3 install transformers sentencepiece torch datasets sacrebleu pyter3 accelerate
    !pip3 install -U scikit-learn scipy matplotlib

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving requirements_task3.txt to requirements_task3 (1).txt
User uploaded file "requirements_task3.txt" with length 1738 bytes
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Collection of all the imports required to run this noteboook

In [3]:
import json
import pyter
import string
import statistics
import pandas as pd
from tqdm import tqdm
from itertools import islice
from sacrebleu.metrics import BLEU
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

Before executing the subsequent notebook cells, please clone all the related files from the [official Github repository](https://github.com/FarhanDhanani/joker-clef-22-FAST-MT). And ensure the mentioned directory structure is set up correctly either on the Google Drive if you are executing on the collab or your local system, in case you are running it locally. 

**DIRECTORY STRUCTURE:** 
```
.
├── JOKER_CLEF_TASK_1.ipynb
├── JOKER_CLEF_TASK_2.ipynb
├── JOKER_CLEF_TASK_3.ipynb
├── requirements_task1.txt
├── requirements_task2.txt
├── requirements_task3.txt
│── Readme.md
├── dataset
│   └── JOKER
│       ├── Task 1
│       │   ├── test
│       │   │   ├── check_format.py
│       │   │   ├── joker_task1_en_test.csv
│       │   │   ├── joker_task1_en_test.json
│       │   │   └── joker_task1_en_testfull.json
│       │   └── train
│       │       ├── FT_Model
│       │       │   ├── FINE_TUNED_MODELS_FOR_TASK1_WILL_SAVE_HERE
|       |       |   ├── TFDistilBertForSequenceClassification
│       │       │   │   ├── finetuned model TFDistilBertForSequenceClassification
│       │       │   ├── TFDistilBertForSequenceClassificationOfConventionalType
│       │       │   │   ├── finetuned model TFDistilBertForSequenceClassificationOfConventionalType
│       │       │   ├── TFDistilBertForSequenceClassificationOfCulturalReferenceType
│       │       │   │   ├── finetuned model TFDistilBertForSequenceClassificationOfCulturalReferenceType
│       │       │   ├── TFDistilBertForSequenceClassificationOfHorizontalVerticalType
│       │       │   │   ├── finetuned model TFDistilBertForSequenceClassificationOfHorizontalVerticalType
│       │       │   ├── TFDistilBertForSequenceClassificationOfManipulationType
│       │       │   │   ├── finetuned model TFDistilBertForSequenceClassificationOfManipulationType
│       │       │   ├── bert-finetuned-ner-JOKER_task1
│       │       │   │   ├── finetuned model ner-JOKER_task1
│       │       │   ├── bert-finetuned-ner-location-JOKER_task1
│       │       │   │   ├── finetuned model location-JOKER_task1
│       │       ├── joker_task1_en_train.csv
│       │       ├── joker_task1_en_train.json
│       │       ├── joker_task1_en_trainfull.json
│       │       ├── train_val_split
│       │       │   ├── train1.csv
│       │       │   ├── train1.json
│       │       │   ├── val1.csv
│       │       │   └── val1.json
│       ├── Task 2
│       │   ├── test
│       │   │   ├── CAMEM-solutions_joker_task2_test_context_aware_data_where_answers_exist.json
│       │   │   ├── DISTIL-solutions_joker_task2_test_context_aware_data_where_answers_exist.json
│       │   │   ├── check_format.py
│       │   │   ├── joker_task2_test.csv
│       │   │   ├── joker_task2_test.json
│       │   │   ├── joker_task2_test_context_aware_data_where_answers_exist.csv
│       │   └── train
│       │       ├── FT_Model
│       │       │   ├── camembert-base-squadFR-fquad-piaf
│       │       │   │   ├── Fine-tuned camembert model will save here
│       │       │   └── distilbert-base-cased-distilled-squad
│       │       │       ├── Fine-tuned distilbert model will save here
│       │       ├── joker_task2_train.csv
│       │       ├── joker_task2_train.json
│       │       ├── joker_task2_train_context_aware_data_where_answers_exist.csv
│       │       └── train_val_split
│       │           ├── train2.csv
│       │           └── val2.csv
│       ├── Task 3
│       │   ├── test
│       │   │   ├── check_format.py
│       │   │   ├── joker_task3_test.csv
│       │   │   └── joker_task3_test.json
│       │   └── train
│       │       ├── Helsinki-NLP-performance.json
│       │       ├── joker_task3_train.csv
│       │       ├── joker_task3_train.json
│       │       ├── t5-base-performance.json
│       │       ├── t5-large-performance.json
│       │       └── t5-small-performance.json
│       ├── csv
│       │   ├── context
│       │   │   ├── c_ref_test.csv
│       │   │   └── c_ref_train.csv
│       │   ├── en-fr_books.csv
│       │   ├── en-fr_bookshop.csv
│       │   ├── en-fr_opensub.csv
│       │   ├── en-fr_ted.csv
│       │   ├── en-fr_wiki.csv
│       │   ├── en-fr_wikipedia.csv
│       │   ├── en_processed
│       │   │   ├── en-fr_books.csv
│       │   │   ├── en-fr_bookshop.csv
│       │   │   ├── en-fr_opensub.csv
│       │   │   ├── en-fr_ted.csv
│       │   │   ├── en-fr_wiki.csv
│       │   │   └── en-fr_wikipedia.csv
│       │   └── 
│       ├── en-fr_books.tmx
│       ├── en-fr_bookshop.tmx
│       ├── en-fr_opensub.tmx
│       ├── en-fr_ted.tmx
│       ├── en-fr_wiki.tmx
│       ├── en-fr_wikipedia.tmx
│       ├── dragons.csv
│       ├── task2_pokemon_en.json
│       └── task2_pokemon_fr.json
├── models
|   |── Downloaded Pre-trained Models will get save here
│   ├── Helsinki-NLP-opus-mt-en-fr
│   │   
│   ├── bert-base-cased
│   │   
│   ├── camembert-base-squadFR-fquad-piaf
│   │   
│   ├── distilbert-base-cased-distilled-squad
│   │   
│   ├── distilbert-base-uncased
│   │   
│   ├── mt5-small
│   │   
│   ├── roberta-base
│   │   
│   └── t5-base
│       
├── .
```

# **TASK 3: TRANSLATE ENTIRE PHRASES CONTAINING WORDPLAY.**

Train data format: List of translated wordplay instances in a JSON format or a CSV file (for manual runs) with the following fields:

    id: a unique wordplay identifier
    en: wordplay text in English (source)
    fr: wordplay text in French (target)

Example:

    [{"id":"pun_724_1","en":"My name is Wade and I'm in swimming pool maintenance.","fr":" Je m\u2019appelle Jacques Ouzy, je m\u2019occupe de l\u2019entretien des piscines."}]

Test data input format: List of wordplay instances to translate in a JSON format or a CSV file (for manual runs) with the following fields:

    id: a unique wordplay identifier
    en: wordplay text in English (source)

Input example:

    [{"id":"het_713","en":"Ever since my mineral extraction facility was converted to parking, I've had a lot on my mine."}]


Test data output format:

List of wordplay instances to be translated in a JSON format or a CSV file (for manual runs) with the following fields:

    RUN_ID: Run ID starting with team_id_ (as registered at the CLEF website)
    MANUAL: Whether the run is manual {0,1}
    id: a unique wordplay identifier
    en: wordplay text in English (source)
    fr: wordplay text in French (target)

Output example:

    [{"RUN_ID":"JCM","MANUAL":1,"id":"pun_724_1","en":"My name is Wade and I'm in swimming pool maintenance.","fr":" Je m\u2019appelle Jacques Ouzy, je m\u2019occupe de l\u2019entretien des piscines."}]


**Evaluation**. Human evaluators will manually annotate the submitted translations according to both subjective measures and according to more concrete features such as whether wordplay exists in the target text, whether it corresponds to the type used in the source text, whether the target text preserves the semantic field, etc.

**Result submission**. Participants should put their run results into the folder Documents created for their user and submit them by email to contact@joker-project.com. The email subject has to be in the format [CLEF TASK 3] TEAM_ID.

## LOADING THE DATASETS

LOADING THE TRAIN DATASET FOR TASK-3

In [4]:
'''
## The train data file were given in the form of csv/json format.
## We have load them into pandas dataframe for further analysis.
'''
path = "/dataset/JOKER/Task 3/train/joker_task3_train.csv"
data_set_3 = pd.read_csv(base_path+path)
data_set_3.head(5)

,id,en,fr
0,pun_11_1,"""A good #deed is never lost. Character is #pro...",Une bonne action n'est jamais perdue. C'est la...
1,pun_11_2,"""A good #deed is never lost. Character is #pro...",On reconnaît la valeur d'un homme à ses action...
2,pun_11_3,"""A good #deed is never lost. Character is #pro...",Quel est le comble pour un notaire? De transme...
3,pun_18_1,"""Dad, I'm cold.""\n""Go stand in the corner, I h...",Où envoie-t-on un vilain petit canard?\nOn l'e...
4,pun_18_2,"""Dad, I'm cold.""\n""Go stand in the corner, I h...","""Papa, j'ai froid.""\n""Va au coin, il fait 90 d..."


THE STRUCTURE OF THE TRAIN DATASET

In [5]:
'''
 ## Determining the structure of the training data set
'''
data_set_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5115 entries, 0 to 5114
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5115 non-null   object
 1   en      5112 non-null   object
 2   fr      5112 non-null   object
dtypes: object(3)
memory usage: 120.0+ KB


LOADING THE TEST DATASET FOR TASK-3

In [6]:
'''
## The test data file were given in the form of csv/json format.
## We have load them into pandas dataframe for further analysis.
'''
path = "dataset/JOKER/Task 3/test/joker_task3_test.csv"
data_set_test_3 = pd.read_csv(base_path+path)
data_set_test_3.head(5)

,id,en
0,hom_1,My boyfriend and I started to date after he ba...
1,hom_2,Some burglars are always looking for windows o...
2,hom_3,OLD PILOTS never die they just go to a higher ...
3,hom_4,Dry cleaners are often pressed for time.
4,hom_5,Cousteau was submerged in his work.


THE STRUCTURE OF THE TEST DATASET

In [7]:
'''
 ## Determining the structure of the testing data set.
'''
data_set_test_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2378 entries, 0 to 2377
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2378 non-null   object
 1   en      2378 non-null   object
dtypes: object(2)
memory usage: 37.3+ KB


## EDA FOR THE LOADED DATASET

TOTAL NO. OF RECORDS IN THE TEST DATASET

In [8]:
'''
 ## The number of total records in the given test data set
'''
len(data_set_test_3)

2378

NUMBER OF MISSING VALUES IN EACH COLUMN of TEST DATASET

In [9]:
'''
 ## Finding out total Number of missing cells in the given test data set
'''
data_set_test_3.isnull().sum()

id    0
en    0
dtype: int64

TOTAL NO. OF RECORDS IN THE TRAIN DATASET

In [10]:
'''
 ## The number of total records in the given training data set
'''
len(data_set_3)

5115

TOTAL NUMBER OF MISSING VALUES IN EACH COLUMNS OF THE TRAIN DATASET

In [11]:
'''
 ## Finding out total Number of missing cells in the given training data set
'''
data_set_3.isnull().sum()

id    0
en    3
fr    3
dtype: int64

EXPLORING VARIABLE ID OF THE TRAIN DATASET 

In [12]:
'''
 ## Finding out the description data related to id variable
'''
data_set_3.id.describe()

count         5115
unique        5115
top       pun_11_1
freq             1
Name: id, dtype: object

EXPLORING EN COLUMNS OF THE TRAIN DATASET

In [13]:
'''
 ## Finding out the description data related to en variable
'''
data_set_3.en.describe()

count                                                  5112
unique                                                 1238
top       Two astronauts who were dating put an end to i...
freq                                                     33
Name: en, dtype: object

NOT ALL VALUES IN THE EN COLUMN ARE UNIQUE IN THE TRAIN DATA SET

The task is to predict the equivalent parallel translation in French for the given text in English. But, there can be multiple different translations possible for a given English text in the French language under different situations. So, ideally, it makes sense because the authors have just duplicated the English text for each corresponding new French translation to tabulate the data in CSV. It's important to note that the translation models are deterministic, and they can never produce two different translations for the same input text. So, our evaluation must get designed to score the models' predictions according to the closest match among all the possible ground truth French translations.

In [14]:
'''
 ## Finding duplicates in the "en" column of the train data set
'''
dups = data_set_3[data_set_3.duplicated(subset=['en'], keep=False)]
dups.sort_values(['en'], ascending=[1])

,id,en,fr
0,pun_11_1,"""A good #deed is never lost. Character is #pro...",Une bonne action n'est jamais perdue. C'est la...
1,pun_11_2,"""A good #deed is never lost. Character is #pro...",On reconnaît la valeur d'un homme à ses action...
2,pun_11_3,"""A good #deed is never lost. Character is #pro...",Quel est le comble pour un notaire? De transme...
3,pun_18_1,"""Dad, I'm cold.""\n""Go stand in the corner, I h...",Où envoie-t-on un vilain petit canard?\nOn l'e...
4,pun_18_2,"""Dad, I'm cold.""\n""Go stand in the corner, I h...","""Papa, j'ai froid.""\n""Va au coin, il fait 90 d..."
...,...,...,...
4576,pun_1228_5,You should never take rocks for granite.,Vous ne devriez jamais prendre la géologie com...
4577,pun_1228_6,You should never take rocks for granite.,un seul hêtre vous manque et tout est des peup...
4580,pun_-1_1,NaN,496
4581,pun_-1_2,NaN,172


FINDING RECORDS IN WHICH EN COLUMN HAS NUMERIC VALUES IN THE TRAIN DATA SET

There are zero number of records in which English coloumn contains a numeric value.

In [15]:
'''
 ## Checking if there exist any invalid numeric value in the "en" column of the training data set
'''
data_set_3[pd.to_numeric(data_set_3['en'], errors='coerce').notnull()]

,id,en,fr


EXPLORING RECORDS WHERE EN COLUMN HAS NULL VALUES IN THE TRAIN DATA SET

In [16]:
'''
 ## Finding if there exist any NaN values in the "en" column of the training data set
'''
data_set_3[data_set_3.en.isnull()]

,id,en,fr
4580,pun_-1_1,NaN,496
4581,pun_-1_2,NaN,172
4659,Italie : par ici la momie !,NaN,NaN


FINDING RECORDS WHERE EN COLUMN HAS "#NOM?" VALUE IN THE TRAIN DATA SET

In [17]:
'''
 ## Checking if there exist any invalid "#NOM?" value in the "en" column of the training data set
'''
data_set_3[data_set_3.en == "#NOM?"]

,id,en,fr
4944,pun_94_1,#NOM?,"- C’est ça pour vous, une communauté qui s’occ..."
4968,pun_97_1,#NOM?,"- Il s'est auto proclamé juge, juré et bourrea..."
5096,pun_102_1,#NOM?,- La maîtresse était une vieille tortue ; nous...


EXPLORING FR COLOUMN OF THE TRAIN DATA SET

In [18]:
'''
 ### Finding out the description data related to fr variable
'''
data_set_3.fr.describe()

count      5112
unique     5060
top       #NOM?
freq         42
Name: fr, dtype: object

NOT ALL VALUES IN THE FR COLUMN ARE UNIQUE IN THE TRAIN DATA SET

Usually, as this task is predicting the parallel translation in french for the given sentence in English. So, ideally there must not be any diplicates in FRENCH, but it appears that dataset holds some duplicates in the french coloumns may be this is due to dirty data or missing values.

In [19]:
'''
 ## Finding duplicates in the "fr" column of the train data set
'''
dups = data_set_3[data_set_3.duplicated(subset=['fr'], keep=False)]
dups.sort_values(['fr'], ascending=[1])

,id,en,fr
175,pun_64_6,'I've been listening to the Brandenburg Concer...,"""J'ai écouté la Campanella, Liebesträume et Da..."
1903,pun_592_3,I've been listening to the Brandenberg Concert...,"""J'ai écouté la Campanella, Liebesträume et Da..."
4141,pun_1058_3,Two wrongs can make a riot.,#NOM?
2018,pun_618_2,"If you sight a whale, it could be a fluke.",#NOM?
2028,pun_625_2,If you work in a bakery you may have to take o...,#NOM?
...,...,...,...
4384,pun_1150_9,When it got stuck in the mud my car had to be ...,X
3056,pun_806_9,One evening King Arthur's men discovered Sir L...,X
321,pun_93_1,"- Call me a taxi. - OK, you're a taxi.",NaN
4658,pun_659_1,Italy : show me the mummy!,NaN


FINDING RECORDS IN WHICH FR COLUMN HAS NUMERIC VALUES IN THE TRAIN DATA SET

There are five number of records in which French coloumn contains a numeric value. The records needs to be filtered out or they will contribute in misleading the classifier.

In [20]:
'''
 ## Checking if there exist any invalid numeric value in the "fr" column of the training data set
'''
data_set_3[pd.to_numeric(data_set_3['fr'], errors='coerce').notnull()]

,id,en,fr
772,pun_182_2,A woman who became queen considered it her cro...,0.351
1720,pun_526_3,I never have understood how you're supposed to...,-415
1979,pun_610_3,If there's one person you don't want to interr...,-423
4580,pun_-1_1,NaN,496
4581,pun_-1_2,NaN,172


EXPLORING RECORDS WHERE FR COLUMN HOLDS NULL VALUES IN THE TRAIN DATA SET

In [21]:
'''
 ## Finding if there exist any NaN values in the "fr" column of the training data set
'''
data_set_3[data_set_3.fr.isnull()]

,id,en,fr
321,pun_93_1,"- Call me a taxi. - OK, you're a taxi.",NaN
4658,pun_659_1,Italy : show me the mummy!,NaN
4659,Italie : par ici la momie !,NaN,NaN


EXPLORING RECORDS WHERE FR COLUMN HOLDS "#NOM?" VALUES IN THE TRAIN DATA SET

In [22]:
'''
 ## Checking if there exist any invalid "#NOM?" value in the "fr" column of the training data set
'''
data_set_3[data_set_3.fr == "#NOM?"]

,id,en,fr
326,pun_93_7,"- Call me a taxi. - OK, you're a taxi.",#NOM?
327,pun_93_8,"- Call me a taxi. - OK, you're a taxi.",#NOM?
568,pun_140_2,A gambling gardener usually hedges his bets.,#NOM?
598,pun_146_3,A husband who thinks he is as solid as a rock ...,#NOM?
615,pun_152_2,A lawyer asked his dentist to give him a retai...,#NOM?
788,pun_187_1,"Act your praise, not your shoe size.",#NOM?
838,pun_193_3,Airline pilots make many friends in high places.,#NOM?
978,pun_221_2,Be the soul supporter of your children.,#NOM?
992,pun_230_1,"Boy, I wish the elevator were working,''said T...",#NOM?
1009,pun_235_2,Businesses selling pleasure boats often have a...,#NOM?


EXPLORING RECORDS WHERE FR COLUMN HOLDS "X" VALUES IN THE TRAIN DATA SET

In [23]:
'''
 ## Checking if there exist any invalid "X" value in the "fr" column of the training data set
'''
data_set_3[data_set_3.fr == "X"]

,id,en,fr
1005,pun_232_12,Bring me a rubber band and make it snappy !,X
1671,pun_509_10,I just had some coffee that was good only for ...,X
3056,pun_806_9,One evening King Arthur's men discovered Sir L...,X
3438,pun_924_5,The Net - surfing bandits always ended an oper...,X
4149,pun_1064_7,Vines are extremely short - tempered. They sna...,X
4384,pun_1150_9,When it got stuck in the mud my car had to be ...,X


## PRE-PROCESSING THE LOADED TRAINING DATA-SET

CRAFTING FILTERED DATASET FOR TRAINING BY REMOVING INVALID VALUES IN THE TRAIN DATA SET

In [24]:
'''
 ## Eradicating invalid values from the training data set.
'''
filtered_data_set_3 = data_set_3[~data_set_3.fr.isnull()]
filtered_data_set_3 = filtered_data_set_3[~pd.to_numeric(filtered_data_set_3['fr'], errors='coerce').notnull()]
filtered_data_set_3 = filtered_data_set_3[~filtered_data_set_3.en.isnull()]
filtered_data_set_3 = filtered_data_set_3[~pd.to_numeric(filtered_data_set_3['en'], errors='coerce').notnull()]
filtered_data_set_3 = filtered_data_set_3[~(filtered_data_set_3.fr == "#NOM?")]
filtered_data_set_3 = filtered_data_set_3[~(filtered_data_set_3.fr == "X")]
filtered_data_set_3 = filtered_data_set_3[~(filtered_data_set_3.en == "#NOM?")]

LENGTH OF FILTERED DATASET FOR TRAINING

In [25]:
'''
 ## The number of total records in the given filtered data set
'''
len(filtered_data_set_3)

5056

GATHERING REMAINING DUPLICATES IN THE FRENCH COLUMN OF THE FILTERED TRAINING DATA SET

In [26]:
'''
 ## Finding duplicates in the "fr" column of the filtered data set
'''
dups = filtered_data_set_3[filtered_data_set_3.duplicated(subset=['fr'], keep=False)]
dups.sort_values(['fr'], ascending=[1])

,id,en,fr
175,pun_64_6,'I've been listening to the Brandenburg Concer...,"""J'ai écouté la Campanella, Liebesträume et Da..."
1903,pun_592_3,I've been listening to the Brandenberg Concert...,"""J'ai écouté la Campanella, Liebesträume et Da..."
5008,pun_417_1,I always knock on the fridge door before I ope...,(en cours de recherche)
5043,pun_283_1,Did you hear about the horse that didn’t want ...,(en cours de recherche)
546,pun_134_1,A doctor's mistakes go six feet under,- Les erreurs médicales vous envoient au parad...
548,pun_135_2,A doctor's mistakes go six feet under ; a dent...,- Les erreurs médicales vous envoient au parad...
1079,pun_257_1,Congratulations,"- « Félicitations, tu as eu ta licence », dit ..."
1080,pun_258_1,"Congratulations ; you graduated,''said Tom dip...","- « Félicitations, tu as eu ta licence », dit ..."
2926,pun_789_8,"OLD STATISTICIANS never die, they just average...","Les statisticiens ne meurent pas, seulement, l..."
2949,pun_791_5,"OLD STATISTICIANS never die, they just get mean.","Les statisticiens ne meurent pas, seulement, l..."


In French, the term "(en cours de recherche)" means "under research" in English. This value indicates that the correct entry for the mentioned record is not known and is under research. So, we will simply treat the under research entries as invalid entries for now and remove all records where this value appears.

In [27]:
'''
 ## Removing records that contains invalid "(en cours de recherche)" value in the "fr" 
 ## column of the filtered training data set
'''
filtered_data_set_3 = filtered_data_set_3[~(filtered_data_set_3.fr == "(en cours de recherche)")]


AGAIN TABULATING DUPLICATES IN THE FRENCH COLUMN OF THE FILTERED TRAINING DATA SET

In [28]:
'''
 ## Finding duplicates in the "fr" column of the filtered data set
'''
dups = filtered_data_set_3[filtered_data_set_3.duplicated(subset=['fr'], keep=False)]
dups.sort_values(['fr'], ascending=[1])

,id,en,fr
175,pun_64_6,'I've been listening to the Brandenburg Concer...,"""J'ai écouté la Campanella, Liebesträume et Da..."
1903,pun_592_3,I've been listening to the Brandenberg Concert...,"""J'ai écouté la Campanella, Liebesträume et Da..."
546,pun_134_1,A doctor's mistakes go six feet under,- Les erreurs médicales vous envoient au parad...
548,pun_135_2,A doctor's mistakes go six feet under ; a dent...,- Les erreurs médicales vous envoient au parad...
1079,pun_257_1,Congratulations,"- « Félicitations, tu as eu ta licence », dit ..."
1080,pun_258_1,"Congratulations ; you graduated,''said Tom dip...","- « Félicitations, tu as eu ta licence », dit ..."
2926,pun_789_8,"OLD STATISTICIANS never die, they just average...","Les statisticiens ne meurent pas, seulement, l..."
2949,pun_791_5,"OLD STATISTICIANS never die, they just get mean.","Les statisticiens ne meurent pas, seulement, l..."
4313,pun_1142_2,When asked by a Health Department official to ...,Quand on à demandé à un représentant de l’insp...
4322,pun_1143_1,When asked by a Health Department official to ...,Quand on à demandé à un représentant de l’insp...


Some french records still contain invalid values. For example, let's have a look at the instance mentioned in the next cell.

In [29]:
filtered_data_set_3[filtered_data_set_3.id == 'pun_503_4']

,id,en,fr
1653,pun_503_4,"I haven't developed my photographs yet, said T...",*368


Plus, some records are multi-line, as shown below.

In [30]:
record_with_multiple_lines = filtered_data_set_3[filtered_data_set_3.id == "pun_30_1"]
print(record_with_multiple_lines.en.astype(str).values.flatten())
print(record_with_multiple_lines.fr.astype(str).values.flatten())

['"So what do you do?"\n"Nothing fancy, just been able to put the food on the table." \n"Okay, but what\'s your profession? "\n"Waiter."']
["« Tu fais quoi dans la vie ? »\n« Rien de compliqué, mais j'ai toujours du pain sur la planche. »\n« Oui, mais tu fais quoi comme travail ? »\n« Je suis boulanger. »"]


Next, we will design a pre-processing function to handle these impurities.

In [31]:
'''
 ## The function remove_punctuations takes a text either in French or English languages in 
 ## its input to pre-process it and removes the punctuations and non-informative characters.
 
 ## INPUT:
 ## A text either in French or English
 
 ## OUTPUT:
 ## processed text with no puntuations
'''
def remove_punctuations(sentence):
    sentence_w_multi_line = " ".join(sentence.splitlines())
    sentence_w_punct = "".join([i.lower() if i not in string.punctuation else " " for i in sentence_w_multi_line])
    sentence = sentence_w_punct.strip()
    #sentence_w_num = ''.join(i for i in sentence_w_punct if not i.isdigit())
    return sentence

GENERATING PRE-PROCESSED TRAINING DATA-SET

In [32]:
'''
 ## Applying remove_punctuations on the copy of filtered_data_set_3
'''
preprpcess_data_set_3 = filtered_data_set_3.copy()
preprpcess_data_set_3.en = preprpcess_data_set_3.en.apply(remove_punctuations)
preprpcess_data_set_3.fr = preprpcess_data_set_3.fr.apply(remove_punctuations)

If we look again inside the generated processed training data set, we will find some empty records. Because before we applied the pre-processing, they were composed of impurities only, and after the pre-processing, those records were flushed off completely. So, now we have to drop these records altogether because they can not help in training deep learning models now.

In [33]:
'''
 ## Finding empty records in the "fr" column of the processed training data set
'''
preprpcess_data_set_3[preprpcess_data_set_3.fr == '']

,id,en,fr
1618,pun_489_5,i got angry when my cell phone battery died m...,
3019,pun_803_27,old math teachers never die they just become ...,


In [34]:
'''
 ## Removing empty records in the "fr" column of the processed training data set
'''
preprpcess_data_set_3[preprpcess_data_set_3.en == '']

,id,en,fr


Removing records which contains white spaces

In [35]:
'''
 ## Removing empty records which have only white spaces in the "fr" column 
 ## of the processed training data set
'''
preprpcess_data_set_3 = preprpcess_data_set_3[~(preprpcess_data_set_3.fr == '')]

EXPLORING THE PRE-PROCESSED DATA-SET

In [36]:
preprpcess_data_set_3

,id,en,fr
0,pun_11_1,a good deed is never lost character is prop...,une bonne action n est jamais perdue c est la...
1,pun_11_2,a good deed is never lost character is prop...,on reconnaît la valeur d un homme à ses action...
2,pun_11_3,a good deed is never lost character is prop...,quel est le comble pour un notaire de transme...
3,pun_18_1,dad i m cold go stand in the corner i hea...,où envoie t on un vilain petit canard on l en...
4,pun_18_2,dad i m cold go stand in the corner i hea...,papa j ai froid va au coin il fait 90 deg...
...,...,...,...
5110,pun_22_1,i think it be thine indeed for thou liest in...,en vérité oui je crois qu elle est à toi ca...
5111,pun_29_1,say is your tardy master now at hand nay ...,parlez votre maître retardataire vous suit il...
5112,pun_95_1,do you mean that you think you can find out th...,voulez vous dire que vous croyez pouvoir trouv...
5113,pun_410_1,hum this fellow might be in s time a great bu...,hum ce monsieur là était peut être en son te...


If we look closer, then still some records possess invalid characters. For example, let's glance at the following.

In [37]:
record_with_multiple_lines = preprpcess_data_set_3[preprpcess_data_set_3.id == "pun_30_1"]
print(record_with_multiple_lines.en.astype(str).values.flatten())
print(record_with_multiple_lines.fr.astype(str).values.flatten())

['so what do you do    nothing fancy  just been able to put the food on the table     okay  but what s your profession     waiter']
['« tu fais quoi dans la vie   » « rien de compliqué  mais j ai toujours du pain sur la planche  » « oui  mais tu fais quoi comme travail   » « je suis boulanger  »']


We need to remove the special characters **'«'** from everywhere in the text and convert the developed, processed data set into a dictionary for better structuring the training of deep learning models. 

Previously, In the EDA, we have noticed that there were multiple duplicate entries in the 'en' column of the data set. The training data-set has this particular trait because there can be many possible translations for a given English text in French under different contexts. Therefore, while designing the CSV the authors of the data set may have duplicated English entries for each corresponding new French translation. We will group such duplicate records while curating the pre-processed data set into a python dictionary.

In [38]:
'''
 ## Converting the processed data-set into a dictionary
'''
translation_dict = {}
for index, row in tqdm(preprpcess_data_set_3.iterrows()):
    en_sent = row.en
    dups = preprpcess_data_set_3[preprpcess_data_set_3.en == en_sent]
    fr_sentences = []
    for fr_sent in dups.fr:
        fr_sent =  fr_sent.replace("« ", "")
        fr_sent = fr_sent.replace(" »", "")
        fr_sent =  fr_sent.replace("«", "")
        fr_sent = fr_sent.replace("»", "")
        fr_sentences.append(fr_sent)
    if(en_sent not in translation_dict):
        translation_dict[en_sent] = {"references":fr_sentences, "translation": ""}

5052it [00:04, 1202.90it/s]


EXPLORING CRAFTED PRE-PROCESSED DATA-SET 

In [39]:
print(json.dumps(translation_dict, indent=4,))

{
    "a good  deed is never lost  character is  property  it is the noblest of possession     double  pun": {
        "references": [
            "une bonne action n est jamais perdue  c est la propri\u00e9t\u00e9 la plus noble des actionnaires",
            "on reconna\u00eet la valeur d un homme \u00e0 ses actions en bourse",
            "quel est le comble pour un notaire  de transmettre toutes ses propri\u00e9t\u00e9s \u00e0 sa descendance"
        ],
        "translation": ""
    },
    "dad  i m cold    go stand in the corner  i hear it s 90 degrees    pun  dadjoke  funny": {
        "references": [
            "o\u00f9 envoie t on un vilain petit canard  on l envoie au coin coin",
            "papa  j ai froid    va au coin  il fait 90 degr\u00e9 l\u00e0 bas",
            "la temp\u00e9rature id\u00e9ale de cuisson des canards et de 90\u00b0  comme les coin coins"
        ],
        "translation": ""
    },
    "dad  i m hungry   hello  hungry  i m dad   pun  dadjoke  funny": {

In [40]:
len(translation_dict.keys())

1185

## PREPARING THE MODELS

### PREIDCTIONS WITH [HELSINKI NLP EN-FR](https://huggingface.co/Helsinki-NLP/opus-mt-en-fr) MACHINE TRANSLATION MODEL

We can apply the [Helsinki-NLP/opus-mt-en-fr](https://huggingface.co/Helsinki-NLP/opus-mt-en-fr) model from the hugging face repository  to generate the parallel translation for each English text from the developed pre-processed data set and evaluate its predictions.

In [41]:
'''
 ## Setuping the pre-trained "Helsinki-NLP/opus-mt-en-fr" from the hugging face repository.
'''
model_path  = "Helsinki-NLP/opus-mt-en-fr" if download_helsinki_NLP_for_translation_generation else "models/Helsinki-NLP-opus-mt-en-fr"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/784k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.28M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/marian/tokenization_marian.py:196: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Downloading:   0%|          | 0.00/287M [00:00<?, ?B/s]

GENERATING PREDICTIONS FOR EACH LISTED ENGLISH TEXT IN THE CREATED PRE-PROCESSING DATA-SET USING [HELSINKI-NLP MODEL](https://huggingface.co/Helsinki-NLP/opus-mt-en-fr) 

In [42]:
'''
 ## Iterating on English texts present in the developed pre-processed data 
 ## set and generating predictions using the downloaded pre-trained model. 
'''
for key in tqdm(translation_dict):
    token_ids = tokenizer.encode(key, return_tensors='pt')
    model_out = model.generate(token_ids)
    output_text = tokenizer.convert_ids_to_tokens(model_out[0])
    special_tokens = [tokenizer.pad_token, tokenizer.eos_token]
    prediction = ("".join(x.replace("▁", " ") for x in output_text if x not in special_tokens)).strip()
    translation_dict[key]["translation"] = prediction

100%|██████████| 1185/1185 [19:14<00:00,  1.03it/s]


SCORING THE GENERATED PREDICTIONS FOR EACH LISTED ENGLISH TEXT IN THE CREATED PRE-PROCESSING DATA-SET BY [HELSINKI-NLP MODEL](https://huggingface.co/Helsinki-NLP/opus-mt-en-fr) 

In [43]:
'''
 ## Iterating on each record present in the developed preprocessed data set to generate a 
 ## BLUE/TER score for each generated prediction from the model against the attended English 
 ## input by comparing them with the correspondence ground truth French version texts.
'''

for key in tqdm(translation_dict):
    bleu = BLEU()
    refs = [[el] for el in translation_dict[key]["references"]] 
    hyp = translation_dict[key]["translation"]
    blue_score = bleu.corpus_score([hyp], refs).score
    ter_score = min([pyter.ter(hyp.split(), ref[0].split()) for ref in refs])
    translation_dict[key]["bleu"] = blue_score
    translation_dict[key]["ter"] = ter_score

100%|██████████| 1185/1185 [00:15<00:00, 76.96it/s] 


SAVING THE SCORED PREDICTIONS BACK TO THE FILE-SYSTEM

In [44]:
'''
 ## Saving the generated predictionns
'''
with open(base_path + "/dataset/JOKER/Task 3/train/Helsinki-NLP-performance.json", 'w') as fp:
    json_object = json.dumps(translation_dict, indent = 4) 
    json.dump(json_object, fp)

EXPLORING THE STRUCTURE OF FINAL PREDICTIONS OUTPUT

In [45]:
list(islice(translation_dict.items(), 2))

[('a good  deed is never lost  character is  property  it is the noblest of possession     double  pun',
  {'references': ['une bonne action n est jamais perdue  c est la propriété la plus noble des actionnaires',
    'on reconnaît la valeur d un homme à ses actions en bourse',
    'quel est le comble pour un notaire  de transmettre toutes ses propriétés à sa descendance'],
   'translation': "un bon acte n'est jamais perdu caractère est la propriété il est le plus noble de possession double jeu de mots",
   'bleu': 10.862721615727713,
   'ter': 0.9375}),
 ('dad  i m cold    go stand in the corner  i hear it s 90 degrees    pun  dadjoke  funny',
  {'references': ['où envoie t on un vilain petit canard  on l envoie au coin coin',
    'papa  j ai froid    va au coin  il fait 90 degré là bas',
    'la température idéale de cuisson des canards et de 90°  comme les coin coins'],
   'translation': "Papa j'ai froid va se tenir dans le coin je l'entends s 90 degrés pun dadjoke drôle",
   'bleu'

AGGREGATING THE PREDICTION SCORES FOR ESTIMATING THE OVERALL MODEL'S PERFORMANCE 

In [46]:
'''
 ## Calculating an average of BLEU and TER scores
'''
blue_scores = []
ter_scores = []
for key in tqdm(translation_dict):
    blue_scores.append(translation_dict[key]["bleu"])
    ter_scores.append(translation_dict[key]["ter"])

print("\nBLEU SCORES AVG", statistics.mean(blue_scores), "TER SCORES AVG", statistics.mean(ter_scores))

100%|██████████| 1185/1185 [00:00<00:00, 1058175.48it/s]


BLEU SCORES AVG 18.439709797319825 TER SCORES AVG 0.8049160534813946


### PREIDCTIONS WITH [GOOGLE T5 BASE](https://huggingface.co/t5-base) MACHINE TRANSLATION MODEL

We can apply the [Google T5 base](https://huggingface.co/t5-base) model from the hugging face repository to generate the parallel translation for each English text from the developed pre-processed data set and evaluate its predictions.

In [47]:
'''
 ## Setuping the pre-trained "t5-base" from the hugging face repository.
'''
model_path  = "t5-base" if download_google_T5_base_for_translation_generation else "models/t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:161: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

GENERATING PREDICTIONS FOR EACH LISTED ENGLISH TEXT IN THE CREATED PRE-PROCESSING DATA-SET USING [GOOGLE-T5 BASE](https://huggingface.co/t5-base) 

In [48]:
'''
 ## Iterating on English texts present in the developed pre-processed data 
 ## set and generating predictions using the downloaded pre-trained model. 
'''
task_prefix = "translate English to French: "
for key in tqdm(translation_dict):
    token_ids = tokenizer.encode(task_prefix+key, return_tensors='pt')
    model_out = model.generate(token_ids)
    output_text = tokenizer.convert_ids_to_tokens(model_out[0])
    special_tokens = [tokenizer.pad_token, tokenizer.eos_token]
    prediction = ("".join(x.replace("▁", " ") for x in output_text if x not in special_tokens)).strip()
    translation_dict[key]["translation"] = prediction

100%|██████████| 1185/1185 [21:14<00:00,  1.08s/it]


SCORING THE GENERATED PREDICTIONS FOR EACH LISTED ENGLISH TEXT IN THE CREATED PRE-PROCESSING DATA-SET BY [GOOGLE-T5 BASE](https://huggingface.co/t5-base)

In [49]:
'''
 ## Iterating on each record present in the developed preprocessed data set to generate a 
 ## BLUE/TER score for each generated prediction from the model against the attended English 
 ## input by comparing them with the correspondence ground truth French version texts.
'''

for key in tqdm(translation_dict):
    bleu = BLEU()
    refs = [[el] for el in translation_dict[key]["references"]] 
    hyp = translation_dict[key]["translation"]
    blue_score = bleu.corpus_score([hyp], refs).score
    ter_score = min([pyter.ter(hyp.split(), ref[0].split()) for ref in refs])
    translation_dict[key]["bleu"] = blue_score
    translation_dict[key]["ter"] = ter_score

100%|██████████| 1185/1185 [00:02<00:00, 500.34it/s] 


SAVING THE SCORED PREDICTIONS BACK TO THE FILE-SYSTEM

In [50]:
'''
 ## Saving the generated predictionns
'''
with open(base_path + "/dataset/JOKER/Task 3/train/t5-base-performance.json", 'w') as fp:
    json_object = json.dumps(translation_dict, indent = 4) 
    json.dump(json_object, fp)

EXPLORING THE STRUCTURE OF FINAL PREDICTIONS OUTPUT

In [51]:
list(islice(translation_dict.items(), 2))

[('a good  deed is never lost  character is  property  it is the noblest of possession     double  pun',
  {'references': ['une bonne action n est jamais perdue  c est la propriété la plus noble des actionnaires',
    'on reconnaît la valeur d un homme à ses actions en bourse',
    'quel est le comble pour un notaire  de transmettre toutes ses propriétés à sa descendance'],
   'translation': "un bon acte n'est jamais perdu le caractère est la propriété c'est",
   'bleu': 14.991106946711685,
   'ter': 0.8125}),
 ('dad  i m cold    go stand in the corner  i hear it s 90 degrees    pun  dadjoke  funny',
  {'references': ['où envoie t on un vilain petit canard  on l envoie au coin coin',
    'papa  j ai froid    va au coin  il fait 90 degré là bas',
    'la température idéale de cuisson des canards et de 90°  comme les coin coins'],
   'translation': 'dad i m cold go stand in the corner i hear it s 90 degrees',
   'bleu': 2.627961710408444,
   'ter': 1.0714285714285714})]

AGGREGATING THE PREDICTION SCORES FOR ESTIMATING THE OVERALL MODEL'S PERFORMANCE 

In [52]:
'''
 ## Calculating an average of BLEU and TER scores
'''
blue_scores = []
ter_scores = []
for key in tqdm(translation_dict):
    blue_scores.append(translation_dict[key]["bleu"])
    ter_scores.append(translation_dict[key]["ter"])

print("\nBLEU SCORES AVG", statistics.mean(blue_scores), "TER SCORES AVG", statistics.mean(ter_scores))

100%|██████████| 1185/1185 [00:00<00:00, 1072561.55it/s]


BLEU SCORES AVG 12.648719376371607 TER SCORES AVG 0.8395713185164666


### PREIDCTIONS WITH [GOOGLE T5 SMALL](https://huggingface.co/t5-small) MACHINE TRANSLATION MODEL

We can apply the [Google T5 small](https://huggingface.co/t5-small) model from the hugging face repository to generate the parallel translation for each English text from the developed pre-processed data set and evaluate its predictions.

In [53]:
'''
 ## Setuping the pre-trained "t5-small" from the hugging face repository.
'''
model_path  = "t5-small" if download_google_T5_small_for_translation_generation else "models/t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:161: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

GENERATING PREDICTIONS FOR EACH LISTED ENGLISH TEXT IN THE CREATED PRE-PROCESSING DATA-SET USING [GOOGLE-T5 SMALL](https://huggingface.co/t5-small) 

In [54]:
'''
 ## Iterating on English texts present in the developed pre-processed data 
 ## set and generating predictions using the downloaded pre-trained model. 
'''
task_prefix = "translate English to French: "
for key in tqdm(translation_dict):
    token_ids = tokenizer.encode(task_prefix+key, return_tensors='pt')
    model_out = model.generate(token_ids)
    output_text = tokenizer.convert_ids_to_tokens(model_out[0])
    special_tokens = [tokenizer.pad_token, tokenizer.eos_token]
    prediction = ("".join(x.replace("▁", " ") for x in output_text if x not in special_tokens)).strip()
    translation_dict[key]["translation"] = prediction

100%|██████████| 1185/1185 [07:32<00:00,  2.62it/s]


SCORING THE GENERATED PREDICTIONS FOR EACH LISTED ENGLISH TEXT IN THE CREATED PRE-PROCESSING DATA-SET BY [GOOGLE-T5 SMALL](https://huggingface.co/t5-small)

In [55]:
'''
 ## Iterating on each record present in the developed preprocessed data set to generate a 
 ## BLUE/TER score for each generated prediction from the model against the attended English 
 ## input by comparing them with the correspondence ground truth French version texts.
'''
for key in tqdm(translation_dict):
    bleu = BLEU()
    refs = [[el] for el in translation_dict[key]["references"]] 
    hyp = translation_dict[key]["translation"]
    blue_score = bleu.corpus_score([hyp], refs).score
    ter_score = min([pyter.ter(hyp.split(), ref[0].split()) for ref in refs])
    translation_dict[key]["bleu"] = blue_score
    translation_dict[key]["ter"] = ter_score

100%|██████████| 1185/1185 [00:02<00:00, 497.19it/s] 


SAVING THE SCORED PREDICTIONS BACK TO THE FILE-SYSTEM

In [56]:
'''
 ## Saving the generated predictionns
'''
with open(base_path + "/dataset/JOKER/Task 3/train/t5-small-performance.json", 'w') as fp:
    json_object = json.dumps(translation_dict, indent = 4) 
    json.dump(json_object, fp)

EXPLORING THE STRUCTURE OF FINAL PREDICTIONS OUTPUT

In [57]:
list(islice(translation_dict.items(), 2))

[('a good  deed is never lost  character is  property  it is the noblest of possession     double  pun',
  {'references': ['une bonne action n est jamais perdue  c est la propriété la plus noble des actionnaires',
    'on reconnaît la valeur d un homme à ses actions en bourse',
    'quel est le comble pour un notaire  de transmettre toutes ses propriétés à sa descendance'],
   'translation': "a bon acte n'est jamais perdu caractère est bien c'est le",
   'bleu': 4.503733751056993,
   'ter': 0.875}),
 ('dad  i m cold    go stand in the corner  i hear it s 90 degrees    pun  dadjoke  funny',
  {'references': ['où envoie t on un vilain petit canard  on l envoie au coin coin',
    'papa  j ai froid    va au coin  il fait 90 degré là bas',
    'la température idéale de cuisson des canards et de 90°  comme les coin coins'],
   'translation': "j'ai m froide allez stand dans le coin j'entend",
   'bleu': 3.056960239296902,
   'ter': 0.9285714285714286})]

AGGREGATING THE PREDICTION SCORES FOR ESTIMATING THE OVERALL MODEL'S PERFORMANCE 

In [58]:
'''
 ## Calculating an average of BLEU and TER scores
'''
blue_scores = []
ter_scores = []
for key in tqdm(translation_dict):
    blue_scores.append(translation_dict[key]["bleu"])
    ter_scores.append(translation_dict[key]["ter"])

print("\nBLEU SCORES AVG", statistics.mean(blue_scores), "TER SCORES AVG", statistics.mean(ter_scores))

100%|██████████| 1185/1185 [00:00<00:00, 573071.63it/s]


BLEU SCORES AVG 11.071665636902528 TER SCORES AVG 0.8550061575503892


### PREIDCTIONS WITH [GOOGLE T5 LARGE](https://huggingface.co/t5-large) MACHINE TRANSLATION MODEL

We can apply the [Google T5 large](https://huggingface.co/t5-large) model from the hugging face repository to generate the parallel translation for each English text from the developed pre-processed data set and evaluate its predictions.

In [59]:
'''
 ## Setuping the pre-trained "t5-large" from the hugging face repository.
'''
model_path  = "t5-large" if download_google_T5_large_for_translation_generation else "models/t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:161: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


Downloading:   0%|          | 0.00/2.75G [00:00<?, ?B/s]

GENERATING PREDICTIONS FOR EACH LISTED ENGLISH TEXT IN THE CREATED PRE-PROCESSING DATA-SET USING [GOOGLE-T5 LARGE](https://huggingface.co/t5-large) 

In [60]:
'''
 ## Iterating on English texts present in the developed pre-processed data 
 ## set and generating predictions using the downloaded pre-trained model. 
'''
task_prefix = "translate English to French: "
for key in tqdm(translation_dict):
    token_ids = tokenizer.encode(task_prefix+key, return_tensors='pt')
    model_out = model.generate(token_ids)
    output_text = tokenizer.convert_ids_to_tokens(model_out[0])
    special_tokens = [tokenizer.pad_token, tokenizer.eos_token]
    prediction = ("".join(x.replace("▁", " ") for x in output_text if x not in special_tokens)).strip()
    translation_dict[key]["translation"] = prediction

100%|██████████| 1185/1185 [1:01:59<00:00,  3.14s/it]


SCORING THE GENERATED PREDICTIONS FOR EACH LISTED ENGLISH TEXT IN THE CREATED PRE-PROCESSING DATA-SET BY [GOOGLE-T5 LARGE](https://huggingface.co/t5-large)

In [61]:
'''
 ## Iterating on each record present in the developed preprocessed data set to generate a 
 ## BLUE/TER score for each generated prediction from the model against the attended English 
 ## input by comparing them with the correspondence ground truth French version texts.
'''
for key in tqdm(translation_dict):
    bleu = BLEU()
    refs = [[el] for el in translation_dict[key]["references"]] 
    hyp = translation_dict[key]["translation"]
    blue_score = bleu.corpus_score([hyp], refs).score
    ter_score = min([pyter.ter(hyp.split(), ref[0].split()) for ref in refs])
    translation_dict[key]["bleu"] = blue_score
    translation_dict[key]["ter"] = ter_score

100%|██████████| 1185/1185 [00:02<00:00, 503.52it/s] 


SAVING THE SCORED PREDICTIONS BACK TO THE FILE-SYSTEM

In [62]:
'''
 ## Saving the generated predictionns
'''
with open(base_path + "/dataset/JOKER/Task 3/train/t5-large-performance.json", 'w') as fp:
    json_object = json.dumps(translation_dict, indent = 4) 
    json.dump(json_object, fp)

EXPLORING THE STRUCTURE OF FINAL PREDICTIONS OUTPUT

In [63]:
list(islice(translation_dict.items(), 2))

[('a good  deed is never lost  character is  property  it is the noblest of possession     double  pun',
  {'references': ['une bonne action n est jamais perdue  c est la propriété la plus noble des actionnaires',
    'on reconnaît la valeur d un homme à ses actions en bourse',
    'quel est le comble pour un notaire  de transmettre toutes ses propriétés à sa descendance'],
   'translation': 'a good deed is never lost character is property it is the noblest of',
   'bleu': 0.0,
   'ter': 1.0}),
 ('dad  i m cold    go stand in the corner  i hear it s 90 degrees    pun  dadjoke  funny',
  {'references': ['où envoie t on un vilain petit canard  on l envoie au coin coin',
    'papa  j ai froid    va au coin  il fait 90 degré là bas',
    'la température idéale de cuisson des canards et de 90°  comme les coin coins'],
   'translation': 'dad i m cold go stand in the corner i hear it s 90 degrees',
   'bleu': 2.627961710408444,
   'ter': 1.0714285714285714})]

AGGREGATING THE PREDICTION SCORES FOR ESTIMATING THE OVERALL MODEL'S PERFORMANCE 

In [64]:
blue_scores = []
ter_scores = []
for key in tqdm(translation_dict):
    blue_scores.append(translation_dict[key]["bleu"])
    ter_scores.append(translation_dict[key]["ter"])

print("\nBLEU SCORES AVG", statistics.mean(blue_scores), "TER SCORES AVG", statistics.mean(ter_scores))

100%|██████████| 1185/1185 [00:00<00:00, 387967.39it/s]


BLEU SCORES AVG 11.901845248043497 TER SCORES AVG 0.8459136450107567


## GENERATING THE SUBMISSION FILE

LOADING THE TEST DATA FOR TASK-3 OF JOKER CLEF

In [65]:
'''
 ## LOADING TEST DATA SET
'''
path = "dataset/JOKER/Task 3/test/joker_task3_test.csv"
data_set_test_3 = pd.read_csv(base_path+path)
data_set_test_3.head(5)

,id,en
0,hom_1,My boyfriend and I started to date after he ba...
1,hom_2,Some burglars are always looking for windows o...
2,hom_3,OLD PILOTS never die they just go to a higher ...
3,hom_4,Dry cleaners are often pressed for time.
4,hom_5,Cousteau was submerged in his work.


Previously, we have noted that for the training data-set Helsinki-NLP model has given the best performance. So, we will use this model to generate the equivalent French translation for the English text provided in the test data.

In [66]:
'''
 ## LOADING THE HELSINKI MODEL
'''
model_path  = "Helsinki-NLP/opus-mt-en-fr" if download_helsinki_NLP_for_translation_generation else "models/Helsinki-NLP-opus-mt-en-fr"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

/usr/local/lib/python3.7/dist-packages/transformers/models/marian/tokenization_marian.py:196: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


GENERATING THE SUBMISION FILE

In [67]:
task_subimission_3 = []
run_id = "non-disclosable-numeric-UUID"
is_manual = 0
for index, row in tqdm(data_set_test_3.iterrows()):
    predictions_dict = {}
    token_ids = tokenizer.encode(row.en, return_tensors='pt')
    model_out = model.generate(token_ids)
    output_text = tokenizer.convert_ids_to_tokens(model_out[0])
    special_tokens = [tokenizer.pad_token, tokenizer.eos_token]
    prediction = ("".join(x.replace("▁", " ") for x in output_text if x not in special_tokens)).strip()
    predictions_dict["RUN_ID"] = run_id
    predictions_dict["MANUAL"] = is_manual
    predictions_dict["id"] = row.id
    predictions_dict["en"] = row.en
    predictions_dict["fr"] = prediction
    task_subimission_3.append(predictions_dict)
    
with open(base_path + "dataset/JOKER/Task 3/test/[CLEF TASK <3>] "+run_id+".json", 'w') as fp:
    json_object = json.dumps(task_subimission_3, indent = 4) 
    json.dump(json_object, fp)

2378it [35:32,  1.12it/s]
